## **Introducción a Spark usando Python** 


![](http://spark.apache.org/images/spark-logo.png)


### La API de Python



Spark está escrito en Scala, que se compila a bytecode de Java, pero puedes escribir código en Python para comunicarte con la máquina virtual de Java a través de una biblioteca llamada `py4j`. Python tiene la API más completa, pero puede ser algo limitante si necesitas utilizar un método que no está disponible o si necesitas escribir un código especializado. La latencia asociada con la comunicación entre Python y la JVM a veces puede hacer que el código se ejecute más lentamente.  

Una excepción a esto es la biblioteca SparkSQL, que tiene un motor de planificación de ejecución que precompila las consultas. Incluso con esta optimización, hay casos en los que el código puede ejecutarse más lentamente que la versión nativa en Scala.  
La recomendación general para el código en PySpark es utilizar los métodos integrados tanto como sea posible y evitar llamadas excesivamente frecuentes (iterativas) a los métodos de Spark. Si necesitas escribir código de alto rendimiento o especializado, intenta hacerlo en Scala.  

Pero bueno, sabemos que Python es increíble, y sus bibliotecas de visualización son mucho mejores. ¡Así que la decisión es tuya!. 


### Objetivos


En este cuaderno, revisaremos los conceptos básicos de Apache Spark y PySpark. Comenzaremos creando el `SparkContext` y el `SparkSession`. Luego, crearemos un `RDD` y aplicaremos algunas transformaciones y acciones básicas. Finalmente, demostraremos los conceptos básicos de `DataFrames` y `SparkSQL`.  

Después de terminar este cuaderno, serás capaz de:  

* Crear el `SparkContext` y el `SparkSession`  
* Crear un `RDD` y aplicar algunas transformaciones y acciones básicas a los `RDDs`  
* Demostrar el uso básico de `DataFrames` y `SparkSQL`  



----


### Configuración



Para este cuaderno, usaremos Python y Spark (PySpark). Estas bibliotecas deberían estar instaladas en tu entorno de trabajo

In [ ]:
# Instalando paquetes requeridos
#!pip install pyspark
!pip install findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
# PySpark es la API de Spark para Python. En este cuaderno, usamos PySpark para inicializar el contexto de Spark. 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

### Ejercicio 1 - Spark Context y Spark Session  



En este ejercicio, crearás el contexto de Spark e inicializarás la sesión de Spark necesaria para `SparkSQL` y `DataFrames`.  
`SparkContext` es el punto de entrada para las aplicaciones de Spark y contiene funciones para crear `RDDs`, como `parallelize()`.  
`SparkSession` es necesario para `SparkSQL` y operaciones con `DataFrame`. 

#### Tarea 1: Creando la sesión y contexto de spark


In [ ]:
# Creación de una sesión de Spark
spark = SparkSession \
    .builder \
    .appName("Ejemplo básico de DataFrames con Python Spark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


In [ ]:
# Creación de una clase de contexto de Spark
sc = spark.sparkContext

### Tarea 2: Inicializar la sesión de Spark  
Para trabajar con *DataFrames*, solo necesitamos verificar que la instancia de la sesión de Spark ha sido creada.


In [ ]:
if 'spark' in locals() and isinstance(spark, SparkSession):
    print("SparkSession está activa y lista para usar.")
else:
    print("SparkSession no está activa. Por favor, crea una SparkSession.")

####  Ejercicio 2: RDDs  
En este ejercicio trabajaremos con *Resilient Distributed Datasets* (RDDs). Los RDDs son la abstracción de datos primitiva de Spark y utilizamos conceptos de programación funcional para crearlos y manipularlos.  


#### Tarea 1: Crear un RDD  
Con fines de demostración, creamos un RDD llamando a `sc.parallelize()`. Creamos un RDD que contiene enteros del 1 al 30.


In [ ]:
data = list(range(1, 31))  # Convertimos range en una lista para compatibilidad con PySpark

# Primer elemento del iterador
print(f"Primer elemento: {data[0]}")
print(f"Tamaño de la lista: {len(data)}")

# Crear el RDD con 4 particiones
xrangeRDD = sc.parallelize(data, numSlices=4)

# Mostrar información del RDD
print(f"Número de particiones: {xrangeRDD.getNumPartitions()}")
print("Primeros 10 elementos del RDD:", xrangeRDD.take(10))

# Mostrar el contenido completo del RDD (solo si el conjunto de datos es pequeño)
print("Contenido del RDD:", xrangeRDD.collect())

#### Tarea 2: Transformaciones



Una transformación es una operación sobre un RDD que da como resultado un nuevo RDD. El RDD transformado se genera rápidamente porque se evalúa de manera *perezosa* (*lazy evaluation*), lo que significa que el cálculo no se realiza inmediatamente cuando se genera el nuevo RDD.  
El RDD contendrá una serie de transformaciones o instrucciones de cálculo, que solo se ejecutarán cuando se llame a una acción.  

En esta transformación, reducimos cada elemento del RDD en 1. Nota el uso de la función *lambda*. Luego filtramos el RDD para que solo contenga elementos menores a 10.



In [ ]:
subRDD = xrangeRDD.map(lambda x: x-1)
filteredRDD = subRDD.filter(lambda x : x<10)

# Mostrar el resultado
print("Primeros 10 elementos de subRDD (después de map):", subRDD.take(10))
print("Elementos de filteredRDD (después de filter):", filteredRDD.collect())


#### Tarea 3: Acciones 



Una transformación devuelve un resultado al *driver*. Ahora aplicamos la acción `collect()` para obtener la salida de la transformación.


In [ ]:
print(filteredRDD.collect())
filteredRDD.count()

####  Tarea 4: Almacenamiento en caché de datos 


Este simple ejemplo muestra cómo crear un RDD y almacenarlo en caché. ¡Observa la mejora de velocidad **10x**!  
Si deseas ver el tiempo de cómputo real, accede a la interfaz de Spark UI en `host:4040`. Notarás que el segundo cálculo tomó mucho menos tiempo.



In [ ]:
import time 

test = sc.parallelize(range(1,50000),4)
test.cache()

t1 = time.time()
# el primer count activará la evaluación del count *y* almacenará en caché
count1 = test.count()
dt1 = time.time() - t1
print("dt1: ", dt1)


t2 = time.time()
# el segundo count opera solo sobre los datos en caché
count2 = test.count()
dt2 = time.time() - t2
print("dt2: ", dt2)

#test.count()

### Ejercicio 3: DataFrames y SparkSQL 


Para trabajar con el potente motor SQL de Apache Spark, necesitas una *Spark Session*.  
Ya la creamos en el primer ejercicio, así que verifiquemos que la sesión sigue activa.


In [ ]:
spark

#### Tarea 1: Crea tu primer Dataframe!



Puedes crear un conjunto de datos estructurado (similar a una tabla de base de datos) en Spark.  
Una vez hecho esto, puedes utilizar herramientas SQL avanzadas para consultar y unir tus *DataFrames*.


In [ ]:
# Descarga los datos primero en un archivo local `people.json`
!curl https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/people.json >> people.json

In [ ]:
# Lee el conjunto de datos en un dataframe de Spark usando la función `read.json()`
df = spark.read.json("people.json").cache()

In [ ]:
# Imprime el DataFrame y su esquema de datos
df.show()
df.printSchema()

In [ ]:
# Registra el DataFrame como una vista SQL temporal
df.createTempView("people")

#### Tarea 2: Explorar los datos usando funciones de DataFrame y SparkSQL  
En esta sección, exploramos los conjuntos de datos utilizando funciones tanto de *DataFrames* como de consultas SQL correspondientes con *SparkSQL*.  
Nota las diferentes formas de lograr la misma tarea.


In [ ]:
# Seleccionar y mostrar columnas básicas de datos

df.select("name").show()
df.select(df["name"]).show()
spark.sql("SELECT name FROM people").show()

In [ ]:
# Realizar filtrado básico

df.filter(df["age"] > 21).show()
spark.sql("SELECT age, name FROM people WHERE age > 21").show()

In [ ]:
# Realizar agregación básica de datos

df.groupBy("age").count().show()
spark.sql("SELECT age, COUNT(age) as count FROM people GROUP BY age").show()

----


### Pregunta 1 - RDDs


Crea un RDD con enteros del 1 al 50.  
Aplica una transformación para multiplicar cada número por 2, obteniendo un RDD que contenga los primeros 50 números pares.  



In [ ]:
# Codigo inicial
# numbers = range(1, 50)
# numbers_RDD = ...
# even_numbers_RDD = numbers_RDD.map(lambda x: ..)

In [ ]:
# Codigo 

Doble-click **aquí** para la solución.

<!-- La respuesta esta abajo:
numbers = range(1, 50) 
numbers_RDD = sc.parallelize(numbers) 
even_numbers_RDD = numbers_RDD.map(lambda x: x * 2)
print( even_numbers_RDD.collect()) 
-->


#### Pregunta 2 - DataFrames y SparkSQL


Similar al archivo `people.json`, ahora lee el archivo `people2.json` en el cuaderno, cárgalo en un *DataFrame* y aplica operaciones SQL para determinar la edad promedio en nuestro archivo `people2`. 


In [ ]:
# codigo inicial
# !curl https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/people2.json >> people2.json
# df = spark.read...
# df.createTempView..
# spark.sql("SELECT ...")

In [ ]:
# Codigo

Haz doble clic **aquí** para ver una pista.

<!-- La pista está abajo:

1. La consulta SQL "SELECT AVG(nombre_columna) FROM..." se puede utilizar para encontrar el valor promedio de una columna.  
2. Otra posible forma es utilizar las operaciones de *DataFrame* `select()` y `mean()`.
-->


Haz doble clic **aquí** para ver la solución.

<!-- La respuesta está abajo:
!curl https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/people2.json >> people2.json
df = spark.read.json("people2.json").cache()
df.createTempView("people2")
result = spark.sql("SELECT AVG(age) from people2")
result.show()
-->


### Pregunta 3 - SparkSession


Cierra la *SparkSession* que creamos para este cuaderno.  


In [ ]:
# Codigo

Haz doble clic **aquí** para ver la solución.

<!-- La respuesta está abajo:

spark.stop() detendrá la sesión de Spark.

-->  
